In [ ]:
# cat and dog classification using CNN
import tensorflow as tf
import tensorflow_datasets as tfds
import os, cv2


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

# loading the dataset
dataset, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True)
class_names = info.features['label'].names



for i, example in enumerate(dataset['train']):
  image, label = example
  save_dir = './cats_vs_dogs/train/{}'.format(class_names[label])
  os.makedirs(save_dir, exist_ok=True)

  filename = save_dir + '/' + "{}_{}.jpg".format(class_names[label], i)
  tf.keras.preprocessing.image.save_img(filename, image.numpy())
  print(filename)


import numpy as np
import cv2
from google.colab.patches import cv2_imshow
img = cv2.imread("/content/cats_vs_dogs/train/cat/cat_100.jpg")
cv2_imshow(img)

img1 = cv2.imread("/content/cats_vs_dogs/train/dog/dog_1.jpg")
cv2_imshow(img1)

datagen = ImageDataGenerator(rescale=1/255, validation_split=0.2, rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, zoom_range=0.10, horizontal_flip=True)

print(datagen)

train_generator = datagen.flow_from_directory(
    '/content/cats_vs_dogs/train', target_size=(150, 150), batch_size=32, class_mode='binary', subset='training')


validation_generator = datagen.flow_from_directory(
    '/content/cats_vs_dogs/train', target_size=(150, 150), batch_size=32, class_mode='binary', subset='validation')

model = Sequential()

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Sequential

# Create a new Sequential model
model = Sequential()

# Add all layers at once
model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(2))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(2))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(2))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# The problematic lines were removed from here
# model.add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(150,150,3)))
# model.add(MaxPooling2D(2))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

# model.add(Conv2D(64,kernel_size=3,activation='relu'))
# model.add(MaxPooling2D(2))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))


# model.add(Conv2D(128,kernel_size=3,activation='relu'))
# model.add(MaxPooling2D(2))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

# model.add(Flatten())
# model.add(Dropout(0.5))
# model.add(Dense(512,activation='relu'))
# model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',patience=3, restore_best_weights=True)
history=model.fit(train_generator,epochs=10,validation_data=validation_generator,callbacks=[early_stop]) # Changed early_stopping to early_stop
history.history

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(train_generator,epochs=10,validation_data=validation_generator,callbacks=[early_stop])
history.history


In [ ]:
# ✅ Step 1: Visualize Training Results
import matplotlib.pyplot as plt

# Extract accuracy and loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 4))

# Plot Training vs Validation Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy', color='blue')
plt.plot(epochs_range, val_acc, label='Validation Accuracy', color='red')
plt.legend()
plt.title('Training and Validation Accuracy')

# Plot Training vs Validation Loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss', color='blue')
plt.plot(epochs_range, val_loss, label='Validation Loss', color='red')
plt.legend()
plt.title('Training and Validation Loss')

plt.show()

# ✅ Step 2: Save the Model
model.save("cats_vs_dogs_classifier.h5")
print("Model saved successfully!")

# ✅ Step 3: Load & Test Model with a New Image
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))  # Resize image
    img_array = image.img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    prediction = model.predict(img_array)

    if prediction[0][0] > 0.5:
        print(f"Predicted: 🐶 Dog ({prediction[0][0]:.2f} confidence)")
    else:
        print(f"Predicted: 🐱 Cat ({1 - prediction[0][0]:.2f} confidence)")

    # Show the image
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# ✅ Step 4: Test the Model with a Sample Image
# Change path accordingly to test a cat or dog image
predict_image('/content/cats_vs_dogs/train/cat/cat_100.jpg')

# ✅ Step 5: Load the Saved Model & Test Again
from tensorflow.keras.models import load_model

loaded_model = load_model("cats_vs_dogs_classifier.h5")
print("Model loaded successfully!")

# Test Again with a Different Image
predict_image('/content/cats_vs_dogs/train/dog/dog_10024.jpg')


In [ ]:
from google.colab import files

uploaded = files.upload()  # This will open a file upload dialog


In [ ]:
import os

# Get the uploaded file name
image_filename = list(uploaded.keys())[0]
image_path = os.path.join("/content/", image_filename)

# Predict the uploaded image
predict_image(image_path)